# Some interesting charts for Covid-19 in England

In [ ]:
import os
import datetime
fetch_date = os.environ.get('TODAY', datetime.datetime.today().strftime('%Y-%m-%d'))
fetch_time = os.environ.get('FETCH_TIME', 'a test run')

In [ ]:
input_filename = 'covid_{}.csv'.format(fetch_date)

In [ ]:
from IPython.core.display import display, HTML

In [ ]:
display(HTML('<p>Data was fetched from coronavirus.data.gov.uk on {}.</p>'.format(fetch_time)))

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)

In [ ]:
df = pd.read_csv(input_filename).dropna().set_index('date').sort_index()

In [ ]:
def plot_chart(df, title):
    ax = df.plot.line(title=title, figsize=(14,8))
    ax.xaxis.set_minor_locator(MultipleLocator(1))
    plt.xticks(rotation=45)
    plt.grid(b=True, which='major', color='#666666', linestyle='-')
    plt.grid(b=True, which='minor', color='#999999', linestyle='-', alpha=0.2)
    ax.set_ylim(ymin=0)
    plt.show()

In [ ]:
df_hospital_last_60 = df[['hospitalCases']].tail(60)
plot_chart(df_hospital_last_60, 'Cases in hospital over last 60 days')


In [ ]:
df['cases 7 day mean'] = df['newCasesByPublishDate'].rolling(7).mean()
df_cases_last_60 = df[['newCasesByPublishDate', 'cases 7 day mean']].tail(60)
plot_chart(df_cases_last_60, 'Cases over last 60 days')

In [ ]:
df['total tests'] = df.apply(lambda x: x['newPillarOneTestsByPublishDate'] + x['newPillarTwoTestsByPublishDate'], axis=1)
df_test_last_60 = df[['newPillarOneTestsByPublishDate', 'newPillarTwoTestsByPublishDate', 'total tests']].tail(60)
plot_chart(df_test_last_60, 'Tests over last 60 days')

In [ ]:
df['percentage cases per test'] = df.apply(lambda x: x['newCasesByPublishDate'] * 100.0 / x['total tests'], axis=1)
df['7 day mean percentage cases per test'] = df['percentage cases per test'].rolling(7).mean()
df_cases_per_test_last_60 = df[['percentage cases per test', '7 day mean percentage cases per test']].tail(60)
plot_chart(df_cases_per_test_last_60, '% Cases per test over last 60 days')

In [ ]:
df_cases_per_test_all = df[['percentage cases per test', '7 day mean percentage cases per test']]
plot_chart(df_cases_per_test_all, 'Cases per test all data')

In [ ]:
df.to_csv('covid-processed-{}.csv'.format(fetch_date))

In [ ]:
display(HTML('<p>Data in these charts: <a href="covid-processed-{}.csv">covid-processed-{}.csv</a></p>'.format(
    fetch_date, fetch_date)))
display(HTML('<p><a href="https://github.com/RobFisher/rfcovid">Source code</a>.</p>'))
